
# <p style="text-align:center"> <font color='darkorange'>**CUNEF**</font>
## <p style="text-align:center"> **TFM - Análisis de sentimiento pólitico en Twitter**
### <p style="text-align:center"> **3. Predicciones con GPT3**</strong><br />
    
<p style="text-align:left">Pablo Mazariegos Reviriego - <font color='orange'>pablo.mazariegos@cunef.edu </font>

En este proyecto de Trabajo Fin de Máster, realizaremos un análisis de sentimiento de los tweets hechos por los 5 candidatos políticos a la presidencia de Madrid durante el período de campaña política que abarcó desde el 12 hasta el 27 de mayo de 2023. Utilizaremos una base de datos recopilada manualmente que contiene los tweets de los candidatos. El objetivo principal de este proyecto es desarrollar modelos de aprendizaje automático que puedan clasificar los tweets según su sentimiento (positivo, negativo o neutral).

El proyecto se organizará en diferentes cuadernos, cada uno enfocado en una etapa específica del proceso:

 1. EDA y Selección/Preparación de variables
 2. Word Cloud y Análisis de viralidad
 3. <font color='darkgreen'> **Predicciones con GPT3**</font>
 4. Otros Modelos

Este cuaderno se enfoca 

  **INDICE:**
 
 - [Importación de Librerias](#0) 
 - [Funciones utilizadas en este notebook](#1) 
 - [Carga de datos](#2)
 - [Creación de Json de los datos](#3)

##  <a name="0"> Importación de Librerias</a>

In [1]:
g

NameError: name 'g' is not defined

In [ ]:
import pandas as pd
import seaborn as sns
import json
import openai
from getpass import getpass
import unidecode
import os

import warnings
warnings.filterwarnings('ignore')

##  <a name="2"> Carga de datos</a>

In [ ]:
API_KEY_OPENAI = getpass('Introduce tu API_KEY de Open AI:')

openai.api_key = API_KEY_OPENAI
os.environ['OPENAI_API_KEY'] = API_KEY_OPENAI

In [ ]:
file_path = '../data/processed/df_sentimiento_final.csv'
df = pd.read_csv(file_path)
df = df.drop("Unnamed: 0", axis=1)
df.head()

##  <a name="3"> Creación de Json de los datos</a>

In [ ]:
# Crea un nuevo data frame con las columnas que necesitas
new_df = df[['post_clean', 'partido']]

In [ ]:
new_df.head()

In [ ]:
new_df['partido'].unique()

In [ ]:
# Obtén un DataFrame que contiene solo las filas duplicadas
duplicated_rows = new_df[new_df.duplicated()]

# Muestra el valor completo de la columna "post_clean" de las filas duplicadas
for index, row in duplicated_rows.iterrows():
    print(row['post_clean'])

In [ ]:
# Obtén un DataFrame que contiene solo las filas duplicadas
duplicated_rows = new_df[new_df.duplicated()]

# Muestra las filas duplicadas
print(duplicated_rows)

## Guardamos los tweets y partido en un Json

In [ ]:
# Crea una lista vacía para almacenar los nuevos datos
data = []

# Itera sobre cada fila del nuevo data frame
for index, row in new_df.iterrows():
    # Crea un diccionario con el formato que necesitas
    entry = {"prompt": row['post_clean'], "completion": row['partido']}
    # Añade el diccionario a la lista
    data.append(entry)

# Finalmente, escribe los datos en un archivo JSON
with open('formatted_tweets_politicos.json', 'w', encoding='utf-8') as f:
    json.dump(data, f, ensure_ascii=False, indent=4)


In [ ]:
# Verificar la existencia del archivo JSON
file_path = 'formatted_tweets_politicos.json'
if os.path.exists(file_path):
    print("El archivo JSON existe.")
else:
    print("El archivo JSON no existe.")

# Leer el contenido del archivo JSON
with open(file_path, 'r', encoding='utf-8') as f:
    json_data = json.load(f)

# Verificar si el archivo JSON está vacío
if len(json_data) > 0:
    print("El archivo JSON no está vacío.")
else:
    print("El archivo JSON está vacío.")

# Imprimir algunos datos del archivo JSON
print("Primeros 5 registros del archivo JSON:")
for entry in json_data[:5]:
    print(entry)

In [ ]:
!openai tools fine_tunes.prepare_data -f formatted_tweets_politicos.json 

In [ ]:
!openai api fine_tunes.create -t "formatted_tweets_politicos_train.jsonl" -v "formatted_tweets_politicos_valid.jsonl" -m text-davinci-003

## Guardamos los tweets y sentimiento en un Json

In [ ]:
# Crea un nuevo data frame con las columnas que necesitas
new_df = df[['post_clean', 'sentimiento']]

In [ ]:
# Crea una lista vacía para almacenar los nuevos datos
data = []

# Itera sobre cada fila del nuevo data frame
for index, row in new_df.iterrows():
    # Crea un diccionario con el formato que necesitas
    entry = {"prompt": row['post_clean'], "completion": row['sentimiento']}
    # Añade el diccionario a la lista
    data.append(entry)

# Finalmente, escribe los datos en un archivo JSON
with open('formatted_tweets_sentimiento.json', 'w', encoding='utf-8') as f:
    json.dump(data, f, ensure_ascii=False, indent=4)


In [ ]:
!openai api fine_tunes.create -t "formatted_tweets_sentimiento_train.jsonl" -v "formatted_tweets_sentimiento_valid.jsonl" -m text-davinci-003